In [1]:
import sqlite3
import pandas as pd
import time
from IPython.core.display import clear_output

In [21]:
DB = "gpudata.db"

def run_query(q):
    with sqlite3.connect(os.getcwd()+'\\'+DB) as conn:
        return pd.read_sql(q,conn)

def run_command(c):
    with sqlite3.connect(DB) as conn:
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        conn.execute(c)
        
def run_inserts(c, values):
    with sqlite3.connect(DB) as conn:
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        conn.execute(c, values) 
        
def show_tables():
    q = '''
    SELECT
        name,
        type
    FROM sqlite_master
    WHERE type IN ("table","view");
    '''
    return run_query(q)

In [22]:
import os
os.getcwd()+'\\'+DB



'C:\\Users\\Seng\\Desktop\\github\\gpu-analytics\\gpudata.db'

In [13]:
#Create the chipsets table
c0 = """
CREATE TABLE chipsets(
    chipset_id INTEGER PRIMARY KEY AUTOINCREMENT,
    chipset_name TEXT
); 
"""

run_command(c0)

In [15]:
#Create the card_specs table
c1 = """
CREATE TABLE card_specs(
    card_id TEXT PRIMARY KEY,
    card_name TEXT,
    series TEXT,
    chipset_id INTEGER,
    memory_in_GB FLOAT,
    core_clock_in_GHz FLOAT,
    ratings INTEGER,
    manufacturer TEXT,
    part_number TEXT,
    interface TEXT,
    memory_type TEXT,
    tdp_in_watts INTEGER,
    fan TEXT,
    sli_support TEXT,
    crossfire_support TEXT,
    hdmi INTEGER DEFAULT 0,
    FOREIGN KEY(chipset_id) REFERENCES chipsets(chipset_id)
); 
"""

run_command(c1)

In [16]:
#Create the card_prices table
c2 = """
CREATE TABLE card_prices(
    card_id TEXT,
    merchant_id INTEGER,    
    datetime FLOAT,
    price FLOAT,
    PRIMARY KEY(card_id, merchant_id, datetime)
    FOREIGN KEY(card_id) REFERENCES card_specs(card_id)
    FOREIGN KEY(merchant_id) REFERENCES merchants(merchant_id)
); 
"""

run_command(c2)

In [17]:
#Create the merchants table
c3 = """
CREATE TABLE merchants(
    merchant_id INTEGER PRIMARY KEY AUTOINCREMENT,
    merchant_name TEXT
); 
"""

run_command(c3)

In [18]:
#Create the benchmarks table
c4 = """
CREATE TABLE benchmarks(
    chipset_id INTEGER PRIMARY KEY,
    passmark_g3d INTEGER,
    passmark_direct_compute INTEGER,
    FOREIGN KEY(chipset_id) REFERENCES chipsets(chipset_id)
); 
"""

run_command(c4)

In [19]:
show_tables()

,name,type
0,sqlite_sequence,table
1,chipsets,table
2,card_specs,table
3,card_prices,table
4,merchants,table
5,benchmarks,table


In [12]:
#Uncomment the lines to remove the table(s).
#run_command('''DROP table chipsets''')
#run_command('''DROP TABLE card_specs;''')
#run_command('''DROP TABLE card_prices;''')
#run_command('''DROP TABLE merchants;''')
#run_command('''DROP TABLE benchmarks;''')

In [23]:
q0 = 'SELECT * FROM chipsets'
chipsets = run_query(q0)
print(len(chipsets))
chipsets.head()

147


,chipset_id,chipset_name
0,1,GeForce GTX 1080 Ti
1,2,GeForce GTX 1060 6GB
2,3,GeForce GTX 1070
3,4,GeForce GTX 1050 Ti
4,5,GeForce GTX 1070 Ti


In [15]:
q1 = 'SELECT * FROM card_specs'
specs = run_query(q1)
print(len(specs))
#specs.dropna()
specs.head()


511


,card_id,card_name,series,chipset_id,memory_in_GB,core_clock_in_GHz,ratings,manufacturer,part_number,interface,memory_type,tdp_in_watts,fan,sli_support,crossfire_support,hdmi
0,Z8cMnQ,Asus STRIX-GTX1080TI-O11G-GAMING,STRIX GAMING OC,1,11.0,1.57,45,Asus,STRIX-GTX1080TI-O11G-GAMING,PCI-Express x16,GDDR5X,250,Yes,Yes,No,2
1,7RKhP6,Gigabyte GV-N1060WF2OC-6GD,WINDFORCE OC 6G,2,6.0,1.58,38,Gigabyte,GV-N1060WF2OC-6GD,PCI-Express x16,GDDR5,120,Yes,No,No,1
2,bpBrxr,NVIDIA 9001G4112520001,,3,8.0,1.51,17,NVIDIA,9001G4112520001,PCI-Express x16,GDDR5,150,Yes,Yes,No,1
3,ttQRsY,MSI GTX 1050 Ti 4G OC,,4,4.0,1.34,9,MSI,GTX 1050 Ti 4G OC,PCI-Express x16,GDDR5,75,Yes,No,No,1
4,ZWnG3C,MSI GTX 1070 TI GAMING 8G,,5,8.0,1.61,2,MSI,GTX 1070 TI GAMING 8G,PCI-Express x16,GDDR5,180,Yes,Yes,No,1


In [6]:
q2 = 'SELECT * FROM card_prices'
prices = run_query(q2)
prices['datetime'] = prices['datetime'].apply(lambda x: time.strftime('%Y-%m-%d', time.localtime(x))) 
prices.head()


,card_id,merchant_id,datetime,price
0,24X2FT,1,2017-09-12,183.53
1,24X2FT,1,2017-11-01,161.69
2,24X2FT,1,2017-11-02,159.85
3,24X2FT,1,2017-11-08,170.04
4,24X2FT,1,2017-11-10,159.85


In [7]:
q3 = 'SELECT * FROM merchants'
merchants = run_query(q3)
merchants.head()

,merchant_id,merchant_name
0,1,Newegg Marketplace
1,2,Newegg Business
2,3,Newegg
3,4,B&H
4,5,PCM


In [9]:
q4 = 'SELECT c.chipset_id, c.chipset_name, b.passmark_g3d, b.passmark_direct_compute FROM benchmarks b INNER JOIN chipsets c ON c.chipset_id = b.chipset_id'
benchmarks = run_query(q4)
benchmarks.sort_values('passmark_g3d', ascending=False).head()

,chipset_id,chipset_name,passmark_g3d,passmark_direct_compute
12,13,Titan Xp,14305,9302
14,15,Titan V,14166,8801
0,1,GeForce GTX 1080 Ti,14032,9417
18,19,Quadro P6000,13556,8501
6,7,GeForce GTX 1080,12287,7966


In [42]:
q_test = '''
SELECT 
    s.chipset_id,
    c.chipset_name,
    s.card_name,
    s.memory_in_GB,
    s.memory_type
FROM card_specs s
INNER JOIN chipsets c ON c.chipset_id = s.chipset_id
'''
input_value = 16

test_table = run_query(q_test)
test_table[test_table['chipset_id'] == input_value]['memory_type'].mode().values[0]

'GDDR5'

In [48]:
specs_query = '''
SELECT
    c.chipset_id,
    c.chipset_name,
    s.memory_in_GB,
    s.memory_type,
    s.tdp_in_watts,	
    s.core_clock_in_GHz
FROM chipsets c
INNER JOIN card_specs s ON s.chipset_id = c.chipset_id

'''
specs = run_query(specs_query)
specs[specs['chipset_id'] == 3]

,chipset_id,chipset_name,memory_in_GB,memory_type,tdp_in_watts,core_clock_in_GHz
2,3,GeForce GTX 1070,8.0,GDDR5,150,1.51
20,3,GeForce GTX 1070,8.0,GDDR5,150,1.56
32,3,GeForce GTX 1070,8.0,GDDR5,180,1.59
44,3,GeForce GTX 1070,8.0,GDDR5,150,1.58
51,3,GeForce GTX 1070,8.0,GDDR5,150,1.59
53,3,GeForce GTX 1070,8.0,GDDR5,150,1.63
73,3,GeForce GTX 1070,8.0,GDDR5,215,1.51
80,3,GeForce GTX 1070,8.0,GDDR5,150,1.52
85,3,GeForce GTX 1070,8.0,GDDR5,150,1.51
108,3,GeForce GTX 1070,8.0,GDDR5,150,1.51


In [77]:
teset = [1, 3, 5, 6]
import numpy as np
even = 'white'
odd = 'grey'
[even if idx%2==0 else odd for idx, i in enumerate(teset)]

['white', 'grey', 'white', 'grey']

In [74]:
ar%2==0

array([False, False], dtype=bool)

In [11]:
merchants_query = '''
SELECT
    c.chipset_id,
    c.chipset_name,
    s.manufacturer,
    p.price,
    p.datetime,
    m.merchant_id,
    m.merchant_name
FROM chipsets c
INNER JOIN card_specs s ON s.chipset_id = c.chipset_id
INNER JOIN card_prices p on p.card_id = s.card_id
INNER JOIN merchants m on m.merchant_id = p.merchant_id
'''
merchants = run_query(merchants_query)

In [12]:
merchants

,chipset_id,chipset_name,manufacturer,price,datetime,merchant_id,merchant_name
0,52,Radeon Pro WX 3100,AMD,183.53,1.505256e+09,1,Newegg Marketplace
1,52,Radeon Pro WX 3100,AMD,161.69,1.509554e+09,1,Newegg Marketplace
2,52,Radeon Pro WX 3100,AMD,159.85,1.509658e+09,1,Newegg Marketplace
3,52,Radeon Pro WX 3100,AMD,170.04,1.510179e+09,1,Newegg Marketplace
4,52,Radeon Pro WX 3100,AMD,159.85,1.510354e+09,1,Newegg Marketplace
5,52,Radeon Pro WX 3100,AMD,159.10,1.510528e+09,1,Newegg Marketplace
6,52,Radeon Pro WX 3100,AMD,152.99,1.512518e+09,1,Newegg Marketplace
7,52,Radeon Pro WX 3100,AMD,160.11,1.513208e+09,1,Newegg Marketplace
8,52,Radeon Pro WX 3100,AMD,160.11,1.513297e+09,1,Newegg Marketplace
9,52,Radeon Pro WX 3100,AMD,160.11,1.513375e+09,1,Newegg Marketplace


In [10]:
df_filtered_1 = merchants[merchants['chipset_id'] == 2]

NameError: name 'merchants' is not defined

In [16]:
merchant_names = df_filtered_1['merchant_name'].unique() 
merchant_names

array(['Newegg Marketplace', 'Newegg', 'Monoprice', 'Newegg Business',
       'OutletPC', 'Walmart', 'SuperBiiz', 'B&H', 'PCM', 'Adorama',
       'Best Buy', 'Dell Small Business', 'Dell'], dtype=object)

In [18]:
for merchant_name in merchant_names: 
    df_filtered_2 = df_filtered_1[df_filtered_1['merchant_name'] == merchant_name]
    data_1 = df_filtered_2.groupby(['datetime'])['price'].mean()	

In [19]:
data_1

datetime
1.470410e+09    297.99
1.470496e+09    297.99
1.472919e+09    299.99
1.473178e+09    299.99
1.473265e+09    299.99
1.473437e+09    299.99
1.473697e+09    299.99
1.473891e+09    299.99
1.474147e+09    299.99
1.474301e+09    299.99
1.474474e+09    299.99
1.474733e+09    299.99
1.474852e+09    299.99
1.474939e+09    279.99
1.474939e+09    299.99
1.475198e+09    299.99
1.475198e+09    279.99
1.475284e+09    279.99
1.475284e+09    299.99
1.475371e+09    299.99
1.475457e+09    299.99
1.475543e+09    269.99
1.475630e+09    274.99
1.475716e+09    269.99
1.475803e+09    269.99
1.475889e+09    279.99
1.476062e+09    299.99
1.476062e+09    279.99
1.476148e+09    279.99
1.476148e+09    299.99
                 ...  
1.505927e+09    327.99
1.505927e+09    320.99
1.506618e+09    331.99
1.506618e+09    331.99
1.507745e+09    338.99
1.507831e+09    338.99
1.508781e+09    338.99
1.512064e+09    331.99
1.513792e+09    331.99
1.515438e+09    335.49
1.515524e+09    331.99
1.520561e+09    422.99
1.